In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df1=pd.read_csv('/kaggle/input/tuberculosis-chest-xrays-shenzhen/shenzhen_metadata.csv')
base_path='/kaggle/input/tuberculosis-chest-xrays-shenzhen/images/images/'
df1['equalized_filename'] = df1['study_id'].apply(lambda x: base_path+x)
df2=pd.read_csv('/kaggle/input/tuberculosis-chest-xrays-montgomery/montgomery_metadata.csv')
base_path='/kaggle/input/tuberculosis-chest-xrays-montgomery/images/images/'
df2['equalized_filename'] = df2['study_id'].apply(lambda x: base_path+x)
df=pd.concat([df1, df2], ignore_index=True)

In [3]:
df['target']=df['findings'].apply(lambda x:0 if x=='normal' else 1)

In [4]:
df.head()

,study_id,sex,age,findings,equalized_filename,gender,target
0,CHNCXR_0001_0.png,Male,45,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
1,CHNCXR_0002_0.png,Male,63,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
2,CHNCXR_0003_0.png,Female,48,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
3,CHNCXR_0004_0.png,Male,58,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
4,CHNCXR_0005_0.png,Male,28,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-04-23 05:05:31.281079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 05:05:31.281171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 05:05:31.381909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
df['target'] = df['target'].astype(str)

In [7]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,study_id,sex,age,findings,equalized_filename,gender,target
0,CHNCXR_0616_1.png,Male,54,PTB in the right upper field,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,1
1,CHNCXR_0609_1.png,Male,26,PTB in the left upper field,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,1
2,CHNCXR_0277_0.png,Male,47,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
3,CHNCXR_0014_0.png,Male,39,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
4,CHNCXR_0549_1.png,Female,63,bilateral PTB,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,1


In [8]:
df.head()

,study_id,sex,age,findings,equalized_filename,gender,target
0,CHNCXR_0616_1.png,Male,54,PTB in the right upper field,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,1
1,CHNCXR_0609_1.png,Male,26,PTB in the left upper field,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,1
2,CHNCXR_0277_0.png,Male,47,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
3,CHNCXR_0014_0.png,Male,39,normal,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,0
4,CHNCXR_0549_1.png,Female,63,bilateral PTB,/kaggle/input/tuberculosis-chest-xrays-shenzhe...,NaN,1


In [9]:
# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=42)

# Set up data generators
batch_size = 32
image_size = (512, 512)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='equalized_filename',
    y_col='target',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Set class_mode to 'binary'
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_dataframe(
    val_df,
    x_col='equalized_filename',
    y_col='target',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Set class_mode to 'binary'
    subset='validation'
)

Found 512 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.


In [10]:
# import cv2
# def histogram_equalization(image_path):
#     # Read the image in grayscale
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
#     # Apply histogram equalization
#     equalized_img = cv2.equalizeHist(img)
    
#     # Display the original and equalized images
#     plt.figure(figsize=(10, 5))
#     plt.subplot(1, 2, 1)
#     plt.imshow(img, cmap='gray')
#     plt.title('Original Image')
    
#     plt.subplot(1, 2, 2)
#     plt.imshow(equalized_img, cmap='gray')
#     plt.title('Equalized Image')
    
#     plt.show()
# histogram_equalization(df['equalized_filename'][100])

In [11]:
!pip install efficientnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 901.2 kB/s eta 0:00:00a 0:00:01


In [12]:
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D




# Train the model


In [13]:
# Load pre-trained EfficientNetB0 model
base_model_e1 = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Optionally freeze the layers of the base model
for layer in base_model_e1.layers:
    layer.trainable = False

# Add custom classification head
model_e1 = Sequential([
    Input(shape=(512, 512, 3)),
    base_model_e1,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Change the number of units according to your task
])

# Compile the model
model_e1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model_e1.summary()

16804768/16804768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnet-b0 (Functional)    │ (None, 16, 16, 1280)   │     4,049,564 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,377,757 (16.70 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 4,049,564 (15.45 MB)

In [14]:
history_efficientNet = model_e1.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1713848885.057825      93 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713848885.110499      93 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


16/16 ━━━━━━━━━━━━━━━━━━━━ 180s 5s/step - accuracy: 0.6430 - loss: 0.6554 - val_accuracy: 0.7188 - val_loss: 0.5533
Epoch 2/20


W0000 00:00:1713848956.884022      91 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 100s 3s/step - accuracy: 0.7287 - loss: 0.5677 - val_accuracy: 0.8750 - val_loss: 0.4765
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 93s 3s/step - accuracy: 0.8037 - loss: 0.4505 - val_accuracy: 0.8438 - val_loss: 0.3906
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step - accuracy: 0.8124 - loss: 0.4142 - val_accuracy: 0.9062 - val_loss: 0.3939
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 95s 3s/step - accuracy: 0.